In [2]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pymongo

In [5]:
    executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
    browser = Browser('chrome', **executable_path, headless=False)

    # create mars data dictionary and pass to mongo db
    mars_data = {}

    ### NASA Mars News
    url_news = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
    response = requests.get(url_news)
    soup = bs(response.text, 'html5lib')
    news_title = soup.find('div', class_='content_title').find('a').text.strip()
    news_preview = soup.find('div', class_='rollover_description_inner').text.strip()

    mars_data['news_title'] = news_title
    mars_data['news_preview'] = news_preview

    print(news_title)
    print(news_preview)

NASA Invites Students to Name Mars 2020 Rover
Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.


In [6]:
    url_image = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(url_image)
    full_image_button = browser.find_by_id("full_image")
    full_image_button.click()
    more_info_button = browser.links.find_by_partial_text("more info")
    more_info_button.click()
    html = browser.html
    image_soup = bs(html, "html.parser")
    image_url = image_soup.find("figure").find("a").find("img")['src']
    featured_image_url = f"https://www.jpl.nasa.gov/{image_url}"

    mars_data['featured_image_url'] = featured_image_url
    print(featured_image_url)

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA16469_hires.jpg


In [7]:
    twitter = 'https://twitter.com/marswxreport?lang=en'
    twitter_html = requests.get(twitter)
    twitter_soup = bs(twitter_html.text, 'html5lib')

    mars_weather = twitter_soup.find('div', class_="js-tweet-text-container").find('p').text
    mars_weather_img = twitter_soup.find('div', class_="AdaptiveMedia-singlePhoto").find('img')['src']

    mars_data['mars_weather'] = mars_weather
    mars_data['mars_weather_img'] = mars_weather_img
    print(mars_weather)
    print(mars_weather_img)

InSight sol 380 (2019-12-21) low -97.9ºC (-144.3ºF) high -20.6ºC (-5.0ºF)
winds from the SSE at 6.3 m/s (14.1 mph) gusting to 21.5 m/s (48.2 mph)
pressure at 6.50 hPa
https://pbs.twimg.com/media/EMaLwsHWsAcbOTG.jpg


In [41]:
    Mars_Facts = 'https://space-facts.com/mars/'
    tables = pd.read_html(Mars_Facts)
    df_Mars_Facts = tables[0]
    df_Mars_Facts.columns = ['Describtion', 'Value']
    df_Mars_Facts = df_Mars_Facts.set_index('Describtion')
    df_Mars_Facts

,Value
Describtion,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [42]:
    df_Mars_Facts = df_Mars_Facts.to_dict()

In [43]:
df_Mars_Facts

{'Value': {'Equatorial Diameter:': '6,792 km',
  'Polar Diameter:': '6,752 km',
  'Mass:': '6.39 × 10^23 kg (0.11 Earths)',
  'Moons:': '2 (Phobos & Deimos)',
  'Orbit Distance:': '227,943,824 km (1.38 AU)',
  'Orbit Period:': '687 days (1.9 years)',
  'Surface Temperature:': '-87 to -5 °C',
  'First Record:': '2nd millennium BC',
  'Recorded By:': 'Egyptian astronomers'}}

In [44]:
    mars_data['Mars_Facts'] = df_Mars_Facts
    print(mars_data['Mars_Facts'])

{'Value': {'Equatorial Diameter:': '6,792 km', 'Polar Diameter:': '6,752 km', 'Mass:': '6.39 × 10^23 kg (0.11 Earths)', 'Moons:': '2 (Phobos & Deimos)', 'Orbit Distance:': '227,943,824 km (1.38 AU)', 'Orbit Period:': '687 days (1.9 years)', 'Surface Temperature:': '-87 to -5 °C', 'First Record:': '2nd millennium BC', 'Recorded By:': 'Egyptian astronomers'}}


In [10]:
    Hemispheres = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(Hemispheres)

    Hemispheres_html = browser.html
    Hemispheres_soup = bs(Hemispheres_html, 'html5lib')

    results = Hemispheres_soup.find_all('a', class_="itemLink")

    links = list()

    for result in results:
        link = result['href']
        url = f"https://astrogeology.usgs.gov{link}"
        print(url)
        links.append(url)
    links = list(set(links))

    hemisphere_image_urls = []
    for link in links:
        Hemispheres_repsonse = requests.get(link)
        soup_Hemispheres = bs(Hemispheres_repsonse.text, 'html5lib')
        img_url = soup_Hemispheres('li')[1].a['href']
        title = soup_Hemispheres.h2.text
        hemisphere_image_url = {'title' : title, 'img_url' : img_url}
        hemisphere_image_urls.append(hemisphere_image_url)

    mars_data['hemisphere_image_urls'] = hemisphere_image_urls
    print(hemisphere_image_urls)

https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced
[{'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.g

In [45]:
print(mars_data)

{'news_title': 'NASA Invites Students to Name Mars 2020 Rover', 'news_preview': "Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.", 'featured_image_url': 'https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA16469_hires.jpg', 'mars_weather': 'InSight sol 380 (2019-12-21) low -97.9ºC (-144.3ºF) high -20.6ºC (-5.0ºF)\nwinds from the SSE at 6.3 m/s (14.1 mph) gusting to 21.5 m/s (48.2 mph)\npressure at 6.50 hPa', 'mars_weather_img': 'https://pbs.twimg.com/media/EMaLwsHWsAcbOTG.jpg', 'Mars_Facts': {'Value': {'Equatorial Diameter:': '6,792 km', 'Polar Diameter:': '6,752 km', 'Mass:': '6.39 × 10^23 kg (0.11 Earths)', 'Moons:': '2 (Phobos & Deimos)', 'Orbit Distance:': '227,943,824 km (1.38 AU)', 'Orbit Period:': '687 days (1.9 years)', 'Surface Temperature:': '-87 to -5 °C', 'First Record:': '2nd millennium BC', 'Recorded By:': 'Egyptian astronomers'}}, 'hemisphere_image_urls': [{'title': 'Valles Marineris Hemisphere Enha

In [46]:
mars_data.keys()

dict_keys(['news_title', 'news_preview', 'featured_image_url', 'mars_weather', 'mars_weather_img', 'Mars_Facts', 'hemisphere_image_urls', '_id'])

In [50]:
mars_data['Mars_Facts']['Value']

{'Equatorial Diameter:': '6,792 km',
 'Polar Diameter:': '6,752 km',
 'Mass:': '6.39 × 10^23 kg (0.11 Earths)',
 'Moons:': '2 (Phobos & Deimos)',
 'Orbit Distance:': '227,943,824 km (1.38 AU)',
 'Orbit Period:': '687 days (1.9 years)',
 'Surface Temperature:': '-87 to -5 °C',
 'First Record:': '2nd millennium BC',
 'Recorded By:': 'Egyptian astronomers'}

In [51]:
mars_data.values()

dict_values(['NASA Invites Students to Name Mars 2020 Rover', "Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.", 'https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA16469_hires.jpg', 'InSight sol 380 (2019-12-21) low -97.9ºC (-144.3ºF) high -20.6ºC (-5.0ºF)\nwinds from the SSE at 6.3 m/s (14.1 mph) gusting to 21.5 m/s (48.2 mph)\npressure at 6.50 hPa', 'https://pbs.twimg.com/media/EMaLwsHWsAcbOTG.jpg', {'Value': {'Equatorial Diameter:': '6,792 km', 'Polar Diameter:': '6,752 km', 'Mass:': '6.39 × 10^23 kg (0.11 Earths)', 'Moons:': '2 (Phobos & Deimos)', 'Orbit Distance:': '227,943,824 km (1.38 AU)', 'Orbit Period:': '687 days (1.9 years)', 'Surface Temperature:': '-87 to -5 °C', 'First Record:': '2nd millennium BC', 'Recorded By:': 'Egyptian astronomers'}}, [{'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}, {'

In [52]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mars_db"] 
mars_collection = mydb['mars_collection']

In [53]:
mars_collection.insert_one(mars_data)